In [1]:
import requests
import json
from langchain.utilities import BashProcess
import os
import time
from tenacity import retry, wait_exponential

In [2]:
#Note: The openai-python library support for Azure OpenAI is in preview.
import os
import openai
openai.api_type = "azure"
openai.api_base = "https://gcrgpt4aoai6c.openai.azure.com/"
openai.api_version = "2023-03-15-preview"
openai.api_key = os.getenv("OPENAI_API_KEY")

# @retry(wait=wait_random_exponential(multiplier=1, max=60))
@retry(wait=wait_exponential(multiplier=1, min=20, max=60))
def call_llm(code):
   prompt = [{"role":"system","content":"You are a coding assistant. Given C code, replace all variables with x1, x2, and so on."},{"role":"user","content":"int main() {\n  // variable declarations\n  int x;\n  int y;\n  // pre-conditions\n  (x = 1);\n  (y = 0);\n  // loop body\n  while ((y < 100000)) {\n    {\n    (x  = (x + y));\n    (y  = (y + 1));\n    }\n\n  }\n  // post-condition\nassert( (x >= y) );\n}"},{"role":"assistant","content":"int main() {\n  // variable declarations\n  int x1;\n  int x2;\n  // pre-conditions\n  (x1 = 1);\n  (x2 = 0);\n  // loop body\n  while ((x2 < 100000)) {\n    {\n    (x1  = (x1 + x2));\n    (x2  = (x2 + 1));\n    }\n\n  }\n  // post-condition\nassert( (x1 >= x2) );\n}"},{"role":"user","content":"int main() {\n  // variable declarations\n  int x;\n  int y;\n  // pre-conditions\n  (x = 1);\n  (y = 0);\n  // loop body\n  while ((y < 1000)) {\n    {\n    (x  = (x + y));\n    (y  = (y + 1));\n    }\n\n  }\n  // post-condition\nassert( (x >= y) );\n}"},{"role":"assistant","content":"int main() {\n  // variable declarations\n  int x1;\n  int x2;\n  // pre-conditions\n  (x1 = 1);\n  (x2 = 0);\n  // loop body\n  while ((x2 < 1000)) {\n    {\n    (x1  = (x1 + x2));\n    (x2  = (x2 + 1));\n    }\n\n  }\n  // post-condition\nassert( (x1 >= x2) );\n}"}]
   prompt.append({"role":"user", "content":f"{code}"})

   response = openai.ChatCompletion.create(
   engine="gpt-4",
   messages = prompt,
   temperature=0.2,
   max_tokens=1000,
   top_p=0.95,
   frequency_penalty=0,
   presence_penalty=0,
   stop=None)
   return response.choices[0].message.content
   

In [3]:
i = 0
files = list(os.listdir("c_benchmark"))
tot_files = len(files)
# tot_files = 1
retry = 0
fail_list = []
while i < tot_files:
    with open(f"c_benchmark/{files[i]}") as f:
        try:
            code = f.read()
            output = call_llm(code)
            with open(f"c_benchmark_anon/{files[i]}", "w") as f:
                f.write(output)
            print(f"{i+1}/{tot_files}")

            i += 1
            
        except Exception as e:
            print(f"FAILED {i} - {str(e)}")
            retry += 1
            if retry == 10:
                fail_list.append(i)
                retry = 0
                continue

1/134
2/134
3/134
4/134
5/134
6/134
7/134
8/134
9/134
10/134
11/134
12/134
13/134
14/134
15/134
16/134
17/134
18/134
19/134
20/134
21/134
22/134
23/134
24/134
25/134
26/134
27/134
28/134
29/134
30/134
31/134
32/134
33/134
34/134
35/134
36/134
37/134
38/134
39/134
40/134
41/134
42/134
43/134
44/134
45/134
46/134
47/134
48/134
49/134
50/134
51/134
52/134
53/134
54/134
55/134
56/134
57/134
58/134
59/134
60/134
61/134
62/134
63/134
64/134
65/134
66/134
67/134
68/134
69/134
70/134
71/134
72/134
73/134
74/134
75/134
76/134
77/134
78/134
79/134
80/134
81/134
82/134
83/134
84/134
85/134
86/134
87/134
88/134
89/134
90/134
91/134
92/134
93/134
94/134
95/134
96/134
97/134
98/134
99/134
100/134
101/134
102/134
103/134
104/134
105/134
106/134
107/134
108/134
109/134
110/134
111/134
112/134
113/134
114/134
115/134
116/134
117/134
118/134
119/134
120/134
121/134
122/134
123/134
124/134
125/134
126/134
127/134
128/134
129/134
130/134
131/134
132/134
133/134
134/134


In [4]:
print(fail_list)

[]


In [5]:
# call_llm("""
# int main() {
#   // variable declarations
#   int x;
#   int y;
#   // pre-conditions
#   (x = 1);
#   (y = 0);
#   // loop body
#   while ((y < 1000)) {
#     {
#     (x  = (x + y));
#     (y  = (y + 1));
#     }

#   }
#   // post-condition
# assert( (x >= y) );
# }
# """)